In [284]:
import pandas as pd
import numpy as np
from datetime import date
from statistics import mean, mode

# Tables

In [285]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [286]:
# tables
filename_careerorglink = "careerorglink.xlsx"
filename_leadercareerlink = "leadercareerlink.xlsx"
filename_orgtree = "orgtree.xlsx"
filename_elected = "positions_elected.xlsx"

In [287]:
col = pd.read_excel(path_tables + filename_careerorglink,dtype="str")
col.shape

(9002, 12)

In [288]:
col.columns

Index(['CareerString', 'CareerDateString_2022', 'IsJob', 'MultipleSubstrings',
       'CareerStartYear', 'CareerStartMonth', 'CareerSubstring',
       'InstitutionType', 'PrimaryInstitution', 'OrgName', 'Position',
       'Notes'],
      dtype='object')

In [289]:
lcl = pd.read_excel(path_tables + filename_leadercareerlink,dtype="str")
lcl.shape

(12617, 3)

In [290]:
lcl.columns

Index(['LeaderID', 'CareerString', 'CareerDateString_2022'], dtype='object')

In [291]:
org = pd.read_excel(path_tables + filename_orgtree,dtype="str")
org.shape

(2368, 19)

In [292]:
org.columns

Index(['InstitutionType', 'OrgType', 'PrimaryInstitution', 'OrgName',
       'PI_Index', 'OrgRank', 'P1', 'P2', 'P3', 'Alias_OrgName',
       'LinkToNext_PI', 'LinkToNext_Org', 'LinkToNext_Year', 'Notes',
       'L1_Index', 'L2_Index', 'L3_Index', 'L4_Index', 'L5_Index'],
      dtype='object')

In [293]:
elected = pd.read_excel(path_tables + filename_elected,dtype="str")
elected.shape

(105, 4)

In [294]:
elected.columns

Index(['PrimaryInstitution', 'OrgName', 'Position', 'IsElected'], dtype='object')

# Queries

In [295]:
path_queries = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 3 queries/"

In [296]:
# filename_leaderjobtransition_alljobs = "leaderjobtransition_alljobs.xlsx"
# filename_leaderjobtransition_ingov = "leaderjobtransition_ingov.xlsx"

In [297]:
filename_leaderjob_electUnelect_inOutgov = "leaderjob_electUnelect_inOutgov.xlsx"
filename_leaderjobtransition_electUnelect_inOutgov = "leaderjobtransition_electUnelect_inOutgov.xlsx"

In [298]:
# copy this file, adapt and run again, if only want ingov

# filename_leaderjob_elect_and_unelect_ingov = "leaderjob_elect_and_unelect_ingov.xlsx"
# filename_leaderjobtransition_electUnelect_ingovOutgov = "leaderjobtransition_electUnelect_ingovOutgov.xlsx"

In [299]:
# trans = pd.read_excel(path_queries + filename_leaderjobtransition,dtype="str")

In [300]:
today = date.today()
print(today)

2023-10-05


# Analysis

In [301]:
path_analysis = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/analysis/"

In [302]:
# subpaths
og_path = "2023.09.01 analysis/"
study0_path = "2023.10.04 Study 0 - research note/"
study1_path = "2023.10.04 Study 1 - political capital/"
study2_path = "2023.10.04 Study 2 - commitment vs control/"
study3_path = "2023.10.04 Study 3 - reds vs experts/"

# Functions

In [303]:
def merge_results(m):
    
    print("\nMerge Results...")
    print("")
    print("\tshape     :",m.shape)
    print("\tleft_only :",m[m["_merge"]=="left_only"].shape)
    print("\tboth      :",m[m["_merge"]=="both"].shape)
    print("\tright_only:",m[m["_merge"]=="right_only"].shape)

In [304]:
# using this on (PI,OrgName) will ensure unique & non-null keys
# using this on a larger df will ensure unique rows and non-null keys, but not unique keys

def unique_non_null_rows(olddf):
    
    df = olddf.copy()
    
    
    ### drop duplicates
    df = df.drop_duplicates(keep="first",ignore_index=True)

    ### drop null rows
    df.dropna(how="all",axis=0,inplace=True)
    
    # drop rows with null PI
    df = df[~(df["PrimaryInstitution"].isna())]
        
    ### drop rows where PI contains stop words
    stop_words_lower = ["uncertain","current","deprecated","please_revise"]
    df = df[ ~ df["PrimaryInstitution"].str.lower().isin(stop_words_lower)]
            

    print("\nUnique Non-null Rows...")
    print("")
    print("\tNon-unique rows:",olddf.shape)
    print("\tUnique rows    :",df.shape)

    ### sort
    df = df.sort_values(["PrimaryInstitution","OrgName"])
    
    return df

# Step. Format Jobs

#### merge col with elected - to identify elected positions

In [305]:
position_key_columns = ['PrimaryInstitution', 'OrgName', 'Position']
elected_columns = position_key_columns + ["IsElected"]

In [306]:
col2 = col.merge(elected[elected_columns],on=position_key_columns,how="outer",indicator=True)
merge_results(col2)


Merge Results...

	shape     : (9033, 14)
	left_only : (7261, 14)
	both      : (1741, 14)
	right_only: (31, 14)


In [307]:
# address this later... whether
# col2[col2["_merge"]=="right_only"][position_key_columns]

In [308]:
col2.columns

Index(['CareerString', 'CareerDateString_2022', 'IsJob', 'MultipleSubstrings',
       'CareerStartYear', 'CareerStartMonth', 'CareerSubstring',
       'InstitutionType', 'PrimaryInstitution', 'OrgName', 'Position', 'Notes',
       'IsElected', '_merge'],
      dtype='object')

In [309]:
# update col with elected positions
col = col2.drop(columns=["_merge"])

In [310]:
col.IsElected.unique()

array([nan, '1'], dtype=object)

#### select jobs that satisfy criterion for transitions 

In [311]:
# IsJob = True
# CareerStartYear can't be null
# PI, OrgName, Position can't be UNCERTAIN

# IsElected = can be anything

# Criteria 1: is a job
select_jobs1 = (col["IsJob"]=="True")

In [312]:
# Criteria 2: has a start year, not UNCERTAIN

select_jobs2 = (col["CareerStartYear"].notna()) & (col["InstitutionType"]!="UNCERTAIN") & (col["PrimaryInstitution"]!="UNCERTAIN") & (col["OrgName"]!="UNCERTAIN") & (col["Position"]!="UNCERTAIN")

In [313]:
# Criteria 3: is InGov

outside_instituiton_types = ['당외곽및사회단체_사회부문(별책)',
                             '당외곽및사회단체_경제부문(별책)',
                             '당외곽및사회단체_사회부문',
                             '당외곽및사회단체_경제부문',
                             '당외곽및사회단체_체육부문',
                             '당외곽및사회단체_근로단체',
                             '당외곽및사회단체_정치부문',
                             '당외곽및사회단체_대외부문',
                             '당외곽및사회단체_종교부문',
                             '국제친선단체',
                             'UNCERTAIN',
                             np.nan]

# select_jobs3 = ~(col["InstitutionType"].isin(outside_instituiton_types))

In [314]:
col.InstitutionType.unique()

array(['당외곽및사회단체_사회부문(별책)', 'UNCERTAIN', '당외곽및사회단체_경제부문(별책)', '정권기관',
       '인민군', nan, '당외곽및사회단체_체육부문', '당외곽및사회단체_근로단체', '노동당',
       '당외곽및사회단체_정치부문', '국제친선단체', '당외곽및사회단체_대외부문', '당외곽및사회단체_사회부문',
       '당외곽및사회단체_경제부문', '당외곽및사회단체_종교부문'], dtype=object)

In [315]:
col.columns

Index(['CareerString', 'CareerDateString_2022', 'IsJob', 'MultipleSubstrings',
       'CareerStartYear', 'CareerStartMonth', 'CareerSubstring',
       'InstitutionType', 'PrimaryInstitution', 'OrgName', 'Position', 'Notes',
       'IsElected'],
      dtype='object')

In [316]:
job_columns = ['CareerString', 'CareerDateString_2022', 'CareerStartYear', 'CareerStartMonth', 'CareerSubstring',
       'InstitutionType', 'PrimaryInstitution', 'OrgName', 'Position',"IsElected"]

In [317]:
# for all CERTAIN jobs, in and out of gov, elected & unelected, select on Criteria 1 & 2
jobs_all = col.loc[select_jobs1 & select_jobs2,job_columns]
jobs_all.shape

(6000, 10)

In [318]:
# for all CERTAIN, non-elected, and ingov jobs, select on Criteria 1 & 2 & 3
# jobs_ingov = col.loc[select_jobs1 & select_jobs2 & select_jobs3,job_columns]
# jobs_ingov.shape

#### descriptive statistics

In [319]:
# not jobs, excluded
col[~select_jobs1].shape

(2302, 13)

In [320]:
# jobs, excluded, either because elected or because CareerStartYear is unknown
col[select_jobs1 & ~select_jobs2].shape

(731, 13)

In [321]:
# jobs, excluded, because not in gov
col[select_jobs1 & select_jobs2 & ~select_jobs3].shape

(1179, 13)

In [322]:
# out of gov + ingov
col[select_jobs1 & select_jobs2 & ~select_jobs3].shape[0] + col[select_jobs1 & select_jobs2 & select_jobs3].shape[0]

6000

In [323]:
# jobs, out of gov + ingov 
col[select_jobs1 & select_jobs2].shape

(6000, 13)

# Step. add Org and Position metadata from orgtree, such as Rank

In [324]:
org_key_columns = ['PrimaryInstitution', 'OrgName']

In [325]:
orgtree_columns = org_key_columns + ['OrgRank', 'P1', 'P2', 'P3']

In [326]:
jobs_all2 = jobs_all.merge(org[orgtree_columns],on=org_key_columns,how="left",indicator=True)
merge_results(jobs_all2)


Merge Results...

	shape     : (6000, 15)
	left_only : (0, 15)
	both      : (6000, 15)
	right_only: (0, 15)


In [327]:
# Good! No left_only means we are matching on all PI-OrgName keys
# jobs2[jobs2["_merge"]=="left_only"]

In [328]:
jobs_all2.head(2)

,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,CareerSubstring,InstitutionType,PrimaryInstitution,OrgName,Position,IsElected,OrgRank,P1,P2,P3,_merge
0,4.15문학창작단 단장,1989.04,1989,4,NaN,당외곽및사회단체_사회부문(별책),4.15문화창작단,NaN,단장,NaN,0,단장,부단장,NaN,both
1,"1989. 4.15문학창작단 단장, 조선작가동맹 통일문학담당 부위원장",NaN,1989,4,4.15 문화창작단 단장,당외곽및사회단체_사회부문(별책),4.15문화창작단,NaN,단장,NaN,0,단장,부단장,NaN,both


In [329]:
# determine PositionRank - i.e., find Position in P1-P3

# row-vectorized formula - meant for use in df.apply()

def determine_Position_Rank(row):
    
    PositionRank = np.nan
    rowPos = row["Position"]

    # print("\n")
    # print(row["Position"],type(row["Position"]),type(rowPos))
    
    # print("P1:",row["P1"])
    if isinstance(row["P1"],str):
        if rowPos in row["P1"].split(","):
            PositionRank = 1
    
    # print("P2:",row["P2"])
    if isinstance(row["P2"],str):
        if rowPos in row["P2"].split(","):
            PositionRank = 2
          
    # print("P3:",row["P3"])
    if isinstance(row["P3"],str):
        if rowPos in row["P3"].split(","):
            PositionRank = 3
            
    # print("PositionRank:",PositionRank)
        
    return PositionRank

In [330]:
jobs_all2["PositionRank"] = jobs_all2.apply(determine_Position_Rank,axis=1)

In [331]:
jobs_all2.columns

Index(['CareerString', 'CareerDateString_2022', 'CareerStartYear',
       'CareerStartMonth', 'CareerSubstring', 'InstitutionType',
       'PrimaryInstitution', 'OrgName', 'Position', 'IsElected', 'OrgRank',
       'P1', 'P2', 'P3', '_merge', 'PositionRank'],
      dtype='object')

In [332]:
new_jobs_columns = ['CareerString', 'CareerDateString_2022', 'CareerStartYear',
       'CareerStartMonth', 'CareerSubstring', 'InstitutionType',
       'PrimaryInstitution', 'OrgName', 'Position', 'IsElected', 'OrgRank', 'PositionRank']

In [333]:
jobs_all2 = jobs_all2[new_jobs_columns]

In [334]:
jobs_all2.shape

(6000, 12)

# Step. merge leadercareerlink & jobs

In [335]:
job_key_columns = ['CareerString', 'CareerDateString_2022']

In [336]:
lcl.head(2)

,LeaderID,CareerString,CareerDateString_2022
0,리선권,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,조경철,故 김정일 국가장의위원회 위원,2011.12


In [337]:
# leaderjoblink - outer join for descriptive statistics
ljob_outer = lcl.merge(jobs_all2,on=job_key_columns,how="outer",indicator=True)
merge_results(ljob_outer)


Merge Results...

	shape     : (13554, 14)
	left_only : (4960, 14)
	both      : (8594, 14)
	right_only: (0, 14)


In [338]:
# no right_only confirms no mismatched career keys in careerorglink
# left-only indicates all of the excluded career items

In [339]:
# leaderjoblink - inner join for the table we will use to make transitions
ljob = lcl.merge(jobs_all2,on=job_key_columns,how="inner",indicator=True)
merge_results(ljob)


Merge Results...

	shape     : (8594, 14)
	left_only : (0, 14)
	both      : (8594, 14)
	right_only: (0, 14)


In [340]:
# instantiate CareerStartDate
ljob["CareerStartDate"]=np.nan

In [341]:
# CareerStartDate when CareerStartMonth is null
selection = ljob["CareerStartMonth"].isna()
ljob.loc[selection,"CareerStartDate"] = ljob[selection].apply(lambda x: str(x["CareerStartYear"]) + "00",axis=1)

In [342]:
# CareerStartDate when CareerStartMonth has 2 digits
selection = (ljob["CareerStartMonth"].notna()) & (ljob.apply(lambda x: len(str(x["CareerStartMonth"])),axis=1)==2)
ljob.loc[selection,"CareerStartDate"] = ljob[selection].apply(lambda x: str(x["CareerStartYear"]) + str(x["CareerStartMonth"]),axis=1)

In [343]:
# CareerStartDate when CareerStartMonth has 1 digit
selection = (ljob["CareerStartMonth"].notna()) & (ljob.apply(lambda x: len(str(x["CareerStartMonth"])),axis=1)==1)
ljob.loc[selection,"CareerStartDate"] = ljob[selection].apply(lambda x: str(x["CareerStartYear"]) + "0" + str(x["CareerStartMonth"]),axis=1)

In [344]:
ljob.columns

Index(['LeaderID', 'CareerString', 'CareerDateString_2022', 'CareerStartYear',
       'CareerStartMonth', 'CareerSubstring', 'InstitutionType',
       'PrimaryInstitution', 'OrgName', 'Position', 'IsElected', 'OrgRank',
       'PositionRank', '_merge', 'CareerStartDate'],
      dtype='object')

In [345]:
ljob_columns = ['LeaderID', 'CareerString', 'CareerDateString_2022','CareerStartYear', 'CareerStartMonth','CareerStartDate','CareerSubstring', 'InstitutionType',
       'PrimaryInstitution', 'OrgName', 'Position','IsElected','OrgRank', 'PositionRank']

In [346]:
ljob = ljob[ljob_columns]

In [347]:
ljob.head(2)

,LeaderID,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,CareerStartDate,CareerSubstring,InstitutionType,PrimaryInstitution,OrgName,Position,IsElected,OrgRank,PositionRank
0,리원일,노동성 상(유임),1999.02,1999,2,199902,NaN,정권기관,내각,노동성,상,NaN,1,1
1,조용원,당 정치국 후보위원,2020.01,2020,1,202001,NaN,노동당,노동당,당중앙위원회_정치국,후보위원,1,2,3


#### descriptive statistics

In [348]:
# unique leaders, overall
len(lcl.LeaderID.unique())

637

In [349]:
# unique leaders, with jobs included
len(ljob.LeaderID.unique())

607

In [350]:
# leaders not having jobs that satisfy our critiria; excluded from analysis
len(lcl.LeaderID.unique())-len(ljob.LeaderID.unique())

30

In [351]:
# job items, after pairing with leaders (some jobs reported for more than one leader)
ljob.shape

(8594, 14)

In [352]:
# unique positions
position_key_columns = ['PrimaryInstitution', 'OrgName','Position']
ljob[position_key_columns].drop_duplicates().shape

(1566, 3)

In [353]:
# unique orgs
org_key_columns = ['PrimaryInstitution', 'OrgName']
ljob[org_key_columns].drop_duplicates().shape

(991, 2)

# Step. calculate time-adjacent transitions for each leader

In [354]:
def calculate_resume_transitions(lc,leaderid):

    t = lc[(lc["LeaderID"]==leaderid) & (lc["CareerStartYear"].notna())]
    t = t.drop_duplicates(["CareerStartYear","PrimaryInstitution","OrgName"],keep="first")
    t = t.sort_values("CareerStartDate",ignore_index=True)
    # print("resume:",t.shape)
    
    tr = t.merge(t,on="LeaderID",how="inner",suffixes=("_1","_2"))
    # sort based on CareerStartDates
    tr = tr.sort_values(["CareerStartDate_1","CareerStartDate_2"])
    
    # drop keys where CareerStartDate_1 <= CareerStartDate_2
    tr = tr[tr.apply(lambda x: x["CareerStartDate_1"] < x["CareerStartDate_2"],axis=1)]
    #tr.shape
    
    # smallest next startdate
    dates = tr[["CareerStartDate_1","CareerStartDate_2"]]
    #dates
    mindate = dates.groupby("CareerStartDate_1",as_index=False).agg({"CareerStartDate_2":"min"})
    mindate.columns = ["CareerStartDate_1","CareerStartDate_2_min"]
    #mindate
    
    tr2 = tr.merge(mindate,on="CareerStartDate_1",how="left")
    #tr2.shape
    
    tr3 = tr2[tr2.apply(lambda x: x["CareerStartDate_2"] == x["CareerStartDate_2_min"],axis=1)]
    # print("transitions:",tr3.shape)
    
    #print(tr3[some_columns])
    
    print(leaderid,tr3.shape[0],"transitions")

    return tr3

#### a good example of many of the problems we need to address

In [355]:
leaderid = "강관주"
resume = calculate_resume_transitions(ljob,leaderid)

강관주 19 transitions


In [356]:
some_columns = ["LeaderID","CareerStartDate_1","PrimaryInstitution_1","OrgName_1","Position_1","CareerStartDate_2","PrimaryInstitution_2","OrgName_2","Position_2"]
resume[some_columns].head()

,LeaderID,CareerStartDate_1,PrimaryInstitution_1,OrgName_1,Position_1,CareerStartDate_2,PrimaryInstitution_2,OrgName_2,Position_2
0,강관주,196100,체육연구원,체육과학연구소,연구원,197300,노동당,당중앙위원회_문화예술부_X과,과장
1,강관주,196100,체육연구원,체육과학연구소,연구원,197300,평양만경대예술단,섭외부,부부장
17,강관주,197300,노동당,당중앙위원회_문화예술부_X과,과장,198600,노동당,당중앙위원회_조직지도부,부부장
18,강관주,197300,평양만경대예술단,섭외부,부부장,198600,노동당,당중앙위원회_조직지도부,부부장
47,강관주,198600,노동당,당중앙위원회_조직지도부,부부장,198800,노동당,당중앙위원회_통일전선부,부부장


In [357]:
leaderids = list(ljob["LeaderID"].unique())

In [358]:
len(leaderids)

607

In [359]:
def compile_leader_transitions(lc):
    
    leaderids = list(lc["LeaderID"].unique())
    
    leader_resumes = []
    
    for leaderid in leaderids:
        leader_resume = calculate_resume_transitions(lc,leaderid)
        leader_resumes.append(leader_resume)
        
    lct = pd.concat(leader_resumes,ignore_index=True)
    
    return lct

In [360]:
transitions = compile_leader_transitions(ljob)

리원일 16 transitions
조용원 10 transitions
정경택 17 transitions
리하일 13 transitions
김창섭 6 transitions
최태복 38 transitions
장창하 2 transitions
최상건 15 transitions
최희태 2 transitions
차희림 7 transitions
김기남 47 transitions
김령성 13 transitions
김정숙 35 transitions
박성철a 69 transitions
김창룡 2 transitions
김인식 2 transitions
리영호 7 transitions
김영일a 22 transitions
주상성 17 transitions
강동윤 6 transitions
김익현 14 transitions
김일성 69 transitions
김일본 47 transitions
박태화 17 transitions
최광 35 transitions
백학림 42 transitions
리종옥 63 transitions
김의순 23 transitions
김중린 48 transitions
리을설 31 transitions
박송봉 4 transitions
권희경 12 transitions
오진우 40 transitions
양형섭 75 transitions
김영주 65 transitions
김원균 8 transitions
김철만 35 transitions
백인준 15 transitions
황순희 23 transitions
최영림 50 transitions
김영남 57 transitions
전병호 40 transitions
계응태 42 transitions
김락희 16 transitions
홍성남 38 transitions
강석숭 21 transitions
김복신 27 transitions
채희정 34 transitions
김용순 47 transitions
김영춘 31 transitions
길재경 9 transitions
강관주 19 transitions
박용석 17 transitions
박봉주

허철만 9 transitions
박명순 10 transitions
최동명 2 transitions
홍영칠 3 transitions
홍승무 4 transitions
현송월 3 transitions
강순남 4 transitions
김성규 5 transitions
김용수 9 transitions
박태성 13 transitions
정상학 0 transitions
김여정 8 transitions
박정천 10 transitions
권영진 3 transitions
리히용 5 transitions
리선권 7 transitions
김영환 7 transitions
우상철 3 transitions
리태섭 0 transitions
리호림 0 transitions
김능오 9 transitions
김조국 0 transitions
강윤석 15 transitions
박광식 0 transitions
박수일 7 transitions
서창룡 0 transitions
장광봉 0 transitions
장기호 2 transitions
전태수 0 transitions
오동일 0 transitions
정인철 0 transitions
김락겸 1 transitions
장길성 2 transitions
리득남 2 transitions
강봉훈 6 transitions
김세복 0 transitions
장춘성 0 transitions
강표영 6 transitions
윤동현 6 transitions
조경철 7 transitions
고길선 3 transitions
김유일 0 transitions
김정식 0 transitions
리정남 3 transitions
박성철b 0 transitions
박창호 4 transitions
서홍찬 11 transitions
송춘섭 8 transitions
신룡만 3 transitions
안금철 4 transitions
김명남 2 transitions
렴철성 2 transitions
리용주 5 transitions
박영호 4 transitions
장혁 3 transitions
전일호 1

#### add OrgAdvance, PositionAdvance variables

In [361]:
transitions["OrgAdvance"] = transitions.apply(lambda x: int(x["OrgRank_1"]) - int(x["OrgRank_2"]),axis=1)

In [362]:
transitions["PositionAdvance"] = transitions.apply(lambda x: int(x["PositionRank_1"]) - int(x["PositionRank_2"]),axis=1)

#### descriptive statistics

In [363]:
# leaders with 1 or more job transition
len(transitions.LeaderID.unique())

539

In [364]:
# leaders with only 1 job; excluded from analysis
len(ljob.LeaderID.unique())-len(transitions.LeaderID.unique())

68

In [365]:
# number of transitions
transitions.shape

(6311, 30)

In [366]:
transition_counts = transitions.groupby("LeaderID")["Position_1"].count()

In [367]:
min(transition_counts)

1

In [368]:
max(transition_counts)

75

In [369]:
mean(transition_counts)

11.708719851576994

In [370]:
mode(transition_counts)

2

#### frequency of leaders by # of job transitions

In [371]:
x_axis = []
transition_count_distribution = []
for i in range(0,max(transition_counts)+1):
    pdf = sum([1 for count in transition_counts if count ==i])
    # print(i,pdf)
    x_axis.append(i) 
    transition_count_distribution.append(pdf)

In [372]:
transition_count_freq = pd.DataFrame({"Count":x_axis,"Frequency":transition_count_distribution})

In [373]:
# filename_transition_count_freq_alljobs = "transition_count_freq_alljobs.xlsx"
filename_transition_count_freq_ingov = "transition_count_freq_ingov.xlsx"

# transition_count_freq.to_excel(path_analysis + current_subpath + filename_transition_count_freq_ingov,index=False)

# Export Query

#### format transitions query for export

In [374]:
transitions.columns

Index(['LeaderID', 'CareerString_1', 'CareerDateString_2022_1',
       'CareerStartYear_1', 'CareerStartMonth_1', 'CareerStartDate_1',
       'CareerSubstring_1', 'InstitutionType_1', 'PrimaryInstitution_1',
       'OrgName_1', 'Position_1', 'IsElected_1', 'OrgRank_1', 'PositionRank_1',
       'CareerString_2', 'CareerDateString_2022_2', 'CareerStartYear_2',
       'CareerStartMonth_2', 'CareerStartDate_2', 'CareerSubstring_2',
       'InstitutionType_2', 'PrimaryInstitution_2', 'OrgName_2', 'Position_2',
       'IsElected_2', 'OrgRank_2', 'PositionRank_2', 'CareerStartDate_2_min',
       'OrgAdvance', 'PositionAdvance'],
      dtype='object')

In [375]:
# omit CareerStartDate_2_min and any other fields we don't need

transitions_columns = ['LeaderID', 'CareerString_1', 'CareerDateString_2022_1',
       'CareerStartYear_1', 'CareerStartMonth_1', 'CareerStartDate_1',
       'CareerSubstring_1', 'InstitutionType_1', 'PrimaryInstitution_1',
        'OrgName_1', 'Position_1', 'IsElected_1', 'OrgRank_1', 'PositionRank_1',
        'CareerString_2', 'CareerDateString_2022_2', 'CareerStartYear_2',
        'CareerStartMonth_2', 'CareerStartDate_2', 'CareerSubstring_2',
        'InstitutionType_2', 'PrimaryInstitution_2', 'OrgName_2', 'Position_2',
        'IsElected_2', 'OrgRank_2', 'PositionRank_2','OrgAdvance','PositionAdvance']

In [376]:
transitions = transitions[transitions_columns]

#### export job & transition queries

In [377]:
# refresh leaderjob query
ljob.to_excel(path_queries + filename_leaderjob_electUnelect_inOutgov,index=False)

In [378]:
# refresh leaderjobtransition query
transitions.to_excel(path_queries + filename_leaderjobtransition_electUnelect_inOutgov,index=False)